In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
#https://github.com/mwv/zca
from zca.zca import ZCA
import pandas as pd
from sklearn.preprocessing import StandardScaler, PowerTransformer

In [ ]:
class ss_yj_Transformer(BaseEstimator, TransformerMixin):
    # add another additional parameter, just for fun, while we are at it
    
    def __init__(self, feature_names=[], additional_param = ""):  
        self.ss_X = StandardScaler()
        self.pt_X = PowerTransformer(standardize=True)
        self.zca_X = zca.ZCA()
        self.ss_y = StandardScaler()
        self.pt_y = PowerTransformer(standardize=True)  
        self.zca_y = zca.ZCA()
        self.feature_names = feature_names
        self.additional_param = additional_param

    def fit(self, X, y = None):
        self.ss_X.fit(X)
        self.pt_X.fit(self.ss_X.transform(X))
        self.zca_X.fit(self.pt_X.transform(self.ss_X.transform(X)))
        if (y is not None):
            self.ss_y.fit(y)
            self.pt_y.fit(self.ss_y.transform(y))
            self.zca_y.fit(self.pt_y.transform(self.ss_y.transform(y)))
        return self

    def transform(self, X, y = None):
        pt_X_ = pd.DataFrame(self.zca_X.fit_transform(self.pt_X.fit_transform(self.ss_X.fit_transform(X))),columns=X.columns,index=X.index)
        if (y is None):
            return pt_X_
        else:
            pt_y_ = pd.DataFrame(self.zca_y.fit_transform(self.pt_y.fit_transform(self.ss_y.fit_transform(X))),columns=Y.columns,index=Y.index)
            return pt_X_, pt_y_
        
    def inverse_transform(self, X, y = None):
        target_X = pd.DataFrame(self.ss_X.inverse_transform(self.pt_X.inverse_transform(self.zca_X.inverse_transform(X))),index=X.index,columns=X.columns)
        if (y is None):
            return target_X
        else:
            target_y = pd.DataFrame(self.ss_Xy.inverse_transform(self.pt_y.inverse_transform(self.zca_y.inverse_transform(y))),index=y.index,columns=y.columns)
            return target_X, target_y
        

In [ ]:
exclude = 'States'

In [ ]:
all_data = pd.read_csv('../data/raw/states.csv')

X = all_data[set(all_data.columns).difference([exclude])].copy()

In [ ]:
scaler = ss_yj_Transformer()
scaler.fit(X)
X_ = scaler.transform(X)

In [71]:
#scale = 1000
random_set = scaler.inverse_transform(pd.DataFrame(np.random.normal(0, 1, size=(len(X.index), len(X.columns))),columns=X.columns,index=X.index))
#random_set = scaler.inverse_transform(pd.DataFrame(np.random.normal(0, 1, size=(scale*len(X.index), len(X.columns))),columns=X.columns))

In [72]:
random_set.describe()

,Poverty,Traf Deaths,Income,Infant Mort,White,Doctors,Unemployed,University,Crime,Population
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,5.000000e+01
mean,12.361012,1.411525,53031.429229,6.763391,82.887735,257.821473,5.313258,27.322814,400.593318,9.307983e+06
std,3.061499,0.445166,9398.450362,1.677814,11.221322,60.291190,1.101751,4.643011,187.598314,1.385701e+07
min,5.398626,0.579216,35748.684350,3.909831,38.171093,166.525993,3.450273,16.124007,133.683440,2.193095e+05
25%,10.769363,1.118095,47273.103996,5.767167,79.939420,212.968965,4.508332,24.349878,275.997582,3.051386e+06
50%,12.219620,1.303393,51340.411807,6.646379,84.663987,241.862231,5.090317,28.117824,349.469938,4.506992e+06
75%,13.524431,1.602271,58763.768316,7.695496,87.941812,278.507066,5.952256,29.934866,533.095742,8.828703e+06
max,22.230282,2.446102,90145.571856,14.360728,100.419896,435.072268,7.750601,37.328926,837.785705,6.685473e+07


In [67]:
X.describe()

,Poverty,Traf Deaths,Income,Infant Mort,White,Doctors,Unemployed,University,Crime,Population
count,50.000000,50.00000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,5.000000e+01
mean,12.732000,1.40440,51985.100000,6.830000,81.96200,260.276000,5.270000,26.940000,407.480000,6.016154e+06
std,2.939002,0.38654,8592.657934,1.338786,11.97458,64.363415,1.253933,4.761152,183.633868,6.663772e+06
min,7.600000,0.76000,37790.000000,4.700000,29.70000,168.800000,3.000000,17.100000,118.000000,5.232260e+05
25%,10.450000,1.14000,46161.250000,5.725000,76.15000,218.425000,4.400000,24.025000,278.250000,1.779632e+06
50%,12.400000,1.37500,50173.000000,6.850000,84.55000,249.050000,5.300000,26.200000,345.500000,4.313980e+06
75%,14.700000,1.62000,56903.000000,7.750000,89.85000,274.800000,6.275000,29.825000,520.000000,6.490796e+06
max,21.200000,2.45000,70545.000000,10.600000,96.40000,469.000000,8.400000,38.100000,788.000000,3.618591e+07
